In [1]:
import argparse
import os
import random
import subprocess
import time
from distutils.util import strtobool
from typing import List

import numpy as np
import pandas as pd
from gym_microrts import microrts_ai
from gym_microrts.envs.vec_env import MicroRTSGridModeVecEnv

In [2]:
folder = "./MicroRTS-Py/gym_microrts/microrts/maps" #folder that contains map setting files (.xml)
map_paths = []
#find all map files in the folder "recursively"
for root, dirs, files in os.walk(folder):
    for file in files:
        if file.endswith(".xml"):
            map_paths.append(os.path.join(root, file))
#read the first line of each map file (.xml) 
#the format is: <rts.PhysicalGameState width="24" height="24">
# categorize map by width and height
maps_by_size = {}
for map_path in map_paths:
    with open(map_path, "r") as f:
        first_line = f.readline()
        width = int(first_line.split('"')[1])
        height = int(first_line.split('"')[3])
        if (width, height) not in maps_by_size:
            maps_by_size[(width, height)] = []
        maps_by_size[(width, height)].append(map_path)
print(maps_by_size.keys())
print(maps_by_size[(16, 16)])
maps_sizes = {} #reverse map of maps_by_size
for size, maps in maps_by_size.items():
    for _map in maps:
        #remove folder path prefix
        map_name = '/'.join(_map.split("/")[4:])
        maps_sizes[map_name] = size
print(maps_sizes.keys())

dict_keys([(32, 32), (4, 4), (16, 12), (24, 24), (15, 14), (8, 8), (9, 8), (64, 64), (14, 12), (16, 8), (10, 10), (128, 128), (128, 96), (128, 112), (96, 128), (12, 12), (16, 16)])
['./MicroRTS-Py/gym_microrts/microrts/maps/16x16/basesWorkers16x16A.xml', './MicroRTS-Py/gym_microrts/microrts/maps/16x16/EightBasesWorkers16x16.xml', './MicroRTS-Py/gym_microrts/microrts/maps/16x16/melee16x16Mixed8.xml', './MicroRTS-Py/gym_microrts/microrts/maps/16x16/melee16x16Mixed12.xml', './MicroRTS-Py/gym_microrts/microrts/maps/16x16/basesWorkers16x16noResources.xml', './MicroRTS-Py/gym_microrts/microrts/maps/16x16/basesWorkers16x16E.xml', './MicroRTS-Py/gym_microrts/microrts/maps/16x16/basesWorkers16x16L.xml', './MicroRTS-Py/gym_microrts/microrts/maps/16x16/basesWorkers16x16G.xml', './MicroRTS-Py/gym_microrts/microrts/maps/16x16/basesWorkers16x16.xml', './MicroRTS-Py/gym_microrts/microrts/maps/16x16/basesWorkers16x16H.xml', './MicroRTS-Py/gym_microrts/microrts/maps/16x16/TwoBasesBarracks16x16.xml', '.

In [3]:
map_name = "basesWorkers16x16.xml"
print(maps_sizes[map_name])

KeyError: 'basesWorkers16x16.xml'

In [4]:
num_bot_envs = 0
env = MicroRTSGridModeVecEnv(
            num_selfplay_envs = 2,
            num_bot_envs=0,
            partial_obs=False,
            max_steps=2000,
            render_theme=2,
            ai2s=[microrts_ai.coacAI for _ in range(num_bot_envs - 6)]
            + [microrts_ai.randomBiasedAI for _ in range(min(num_bot_envs, 2))]
            + [microrts_ai.lightRushAI for _ in range(min(num_bot_envs, 2))]
            + [microrts_ai.workerRushAI for _ in range(min(num_bot_envs, 2))],
            map_paths=["maps/16x16/basesWorkers16x16.xml"],
            reward_weight=np.array([10.0, 1.0, 1.0, 0.2, 1.0, 4.0]),
            cycle_maps="maps/16x16/basesWorkers16x16.xml", #train map cycle
        )

removing /home/whitefox/user/university_works/DRL-Final-MicroRTS/2025-DRL-Final/MicroRTS-Py/gym_microrts/microrts/microrts.jar...
building /home/whitefox/user/university_works/DRL-Final-MicroRTS/2025-DRL-Final/MicroRTS-Py/gym_microrts/microrts/microrts.jar...
/home/whitefox/user/university_works/DRL-Final-MicroRTS/2025-DRL-Final/MicroRTS-Py


Note: Some input files use or override a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.


adding dependency hamcrest-all-1.3.jar
adding dependency jdom.jar
adding dependency junit-4.12.jar
adding dependency minimal-json-0.9.4.jar
added manifest
adding: JDOMAbout$Author.class(in = 480) (out= 320)(deflated 33%)
adding: JDOMAbout$Info.class(in = 3001) (out= 1678)(deflated 44%)
adding: JDOMAbout.class(in = 2063) (out= 1105)(deflated 46%)
adding: LICENSE-junit.txt(in = 11376) (out= 4182)(deflated 63%)
adding: LICENSE.txt(in = 1509) (out= 801)(deflated 46%)
ignoring entry META-INF/
adding: META-INF/maven/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/com.eclipsesource.minimal-json/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/com.eclipsesource.minimal-json/minimal-json/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/com.eclipsesource.minimal-json/minimal-json/pom.xml(in = 4304) (out= 1154)(deflated 73%)
adding: META-INF/maven/com.eclipsesource.minimal-json/minimal-json/pom.properties(in = 152) (out= 131)(deflated 13%)
adding: META-INF/maven/com.thoughtworks.qdox/(i

In [5]:
def generate_random_action(h, w, attack_range=7):
    num_tiles = h * w
    action_vector = []

    for _ in range(num_tiles):
        action_type = np.random.randint(0, 6)

        move_param = harvest_param = return_param = produce_dir = 0
        produce_type = 0
        attack_param = 0

        if action_type == 1:
            move_param = np.random.randint(0, 4)
        elif action_type == 2:
            harvest_param = np.random.randint(0, 4)
        elif action_type == 3:
            return_param = np.random.randint(0, 4)
        elif action_type == 4:
            produce_dir = np.random.randint(0, 4)
            produce_type = np.random.randint(0, 7)
        elif action_type == 5:
            attack_param = np.random.randint(0, attack_range ** 2)

        tile_action = [
            action_type,
            move_param,
            harvest_param,
            return_param,
            produce_dir,
            produce_type,
            attack_param
        ]
        action_vector.extend(tile_action)

    return np.array(action_vector, dtype=np.int32)

In [6]:
from collections import namedtuple
'''
Source Unit	[0,hxw-1]: the location of the unit selected to perform an action
Action Type	[0,5]: NOOP, move, harvest, return, produce, attack
Move Parameter	[0,3]: north, east, south, west
Harvest Parameter [0,3]: north, east, south, west
Return Parameter [0,3]: north, east, south, west
Produce Direction Parameter	[0,3]: north, east, south, west
Produce Type Parameter	[0,6]: resource, base, barrack, worker, light, heavy, ranged
Relative Attack Position [0,ar^2-1]: the relative location of the unit that will be attacked
'''
class Action_Generator:
    def __init__(self, h, w):
        # coordinate start from left-top corner
        self.h = h
        self.w = w
        self.action_plane = np.full((h, w, 7), [0,0,0,0,0,0,0])
    def flatten_action(self): #we need to submit a 1-d array to the environment position : [(0,0), (0,1), (0,2)...,(1, 0)..., (w-1, h-1)]. Weird, order by y
        ret = np.full((self.h*self.w, 7), [0,0,0,0,0,0,0])
        for x in range(self.w):
            for y in range(self.h):
                ret[x * self.w + y] = self.action_plane[y, x]
        return ret

    def reset_action(self):
        self.action = np.full((self.h, self.w), [0,0,0,0,0,0,0])
    
    def perform_on_grid(self, x, y, action):
        #assert action is a list/array of length 7
        self.action_plane[y][x] = action

In [15]:
from collections import namedtuple
'''
Hit Points	5	0, 1, 2, 3, ≥ 4
Resources	5	0, 1, 2, 3, ≥ 4
Owner	3	-,player 1, player 2
Unit Types	8	-, resource, base, barrack, worker, light, heavy, ranged
Current Action	6	-, move, harvest, return, produce, attack
Terrain	2	free, wall
'''
grid_data = namedtuple("grid_data", "hit_points, resources, owner, unit_types, current_action, terrain")

class Obs_Parser:
    unit_type_dict = {0: "None", 1: "resource", 2: "base", 3: "barrack", 4: "worker", 5: "light", 6: "heavy", 7: "ranged"}
    action_dict = {0: "None", 1: "move", 2: "harvest", 3: "return", 4: "produce", 5: "attack"}

    def __init__(self, vec_obs):
        self.vec_obs = vec_obs #shape: (num_selfplay_env, h, w, 29)
        self.num_env, self.h, self.w, self.c = vec_obs.shape
        self.obs = [] #for each env, parse h*w grid_data
    def parse_feature(self, feature):
        return grid_data(np.argmax(feature[0:5]), np.argmax(feature[5:10]), np.argmax(feature[10:13]), np.argmax(feature[13:21]), np.argmax(feature[21:27]), np.argmax(feature[27:29]))

    def parse(self, vec_obs): #parse 
        self.vec_obs = vec_obs
        self.obs = []
        for e in range(self.num_env):
            plane_data = [[] for _ in range(self.h)]
            for i in range(self.h):
                for j in range(self.w):
                    plane_data[i].append(self.parse_feature(vec_obs[e][i][j]))
            self.obs.append(plane_data)
        return self.obs

In [36]:
act1 = [2,0,3,0,0,0,0]
act2 = [1,2,0,0,0,0,0]
act_none = [0,0,0,0,0,0,0]

In [24]:
action_gen = Action_Generator(16,16)

In [50]:

action = []
action_gen.perform_on_grid(1 ,1 ,act1)
action.append(action_gen.flatten_action())
action_gen.perform_on_grid(1 ,1 ,act_none)
action.append(action_gen.flatten_action())
action_gen.perform_on_grid(1 ,1 ,act2)
action.append(action_gen.flatten_action())
print(action[0][15:18])
print(action[1][15:18])

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [2 0 3 0 0 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]


In [61]:
obs = env.reset()
parser = Obs_Parser(obs)
done = False
step = 0

while step <= 20:
    env.get_action_mask()
    if step == 0:
        actions = np.stack([action[0] for _ in range(2)], axis=0)
    else:
        actions = np.stack([action[1] for _ in range(2)], axis=0)
    obs, reward, done, info = env.step(actions)
    print(reward)
    print(obs[0][1][1])
    print(parser.parse(obs)[0][0][0])
    print(parser.parse(obs)[0][0][1])
    print(parser.parse(obs)[0][1][0])
    print(parser.parse(obs)[0][1][1])
    env.render()
    step += 1



[1. 0.]
[0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0]
grid_data(hit_points=1, resources=4, owner=0, unit_types=1, current_action=0, terrain=0)
grid_data(hit_points=0, resources=0, owner=0, unit_types=0, current_action=0, terrain=0)
grid_data(hit_points=1, resources=4, owner=0, unit_types=1, current_action=0, terrain=0)
grid_data(hit_points=1, resources=0, owner=1, unit_types=4, current_action=2, terrain=0)
[0. 0.]
[0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0]
grid_data(hit_points=1, resources=4, owner=0, unit_types=1, current_action=0, terrain=0)
grid_data(hit_points=0, resources=0, owner=0, unit_types=0, current_action=0, terrain=0)
grid_data(hit_points=1, resources=4, owner=0, unit_types=1, current_action=0, terrain=0)
grid_data(hit_points=1, resources=0, owner=1, unit_types=4, current_action=2, terrain=0)
[0. 0.]
[0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0]
grid_data(hit_points=1, resources=4, owner=0, unit_types=1, current_action=0, terrai

In [12]:
parser = Obs_Parser(obs)
print(parser.parse(obs)[0][1][1])

grid_data(hit_points=1, resources=0, owner=1, unit_types=4, current_action=2, terrain=0)
